In [1]:
!pip install selenium

     |████████████████████████████████| 904 kB 5.4 MB/s eta 0:00:01


In [213]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time, os

chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

url = 'https://store.steampowered.com/search/?filter=topsellers'

#user_agent = {'User-agent': 'Mozilla/5.0'}
driver = webdriver.Chrome(chromedriver)
driver.get(url)

In [146]:
soup = BeautifulSoup(driver.page_source, 'html5lib')

In [136]:
table = soup.find('div', id='search_resultsRows')
table
rows = [row for row in table.find_all('a')]

AttributeError: 'NoneType' object has no attribute 'find_all'

In [49]:
len(rows)

50

In [50]:
#this was sufficient to scroll to the bottom of the top sellers page for steam, pulling code from scraping list
for i in range(500):
    #Scroll
    driver.execute_script(
        "window.scrollTo(0, document.documentElement.scrollHeight);" #Alternatively, document.body.scrollHeight
    )
    time.sleep(1)

In [53]:
soup = BeautifulSoup(driver.page_source, 'html5lib')

In [54]:
#successfully scraped 16.5k elements, hopefully all games.
table = soup.find('div', id='search_resultsRows')
table
rows = [row for row in table.find_all('a')]
rows[0]
len(rows)

16700

In [55]:
#pulling code created in 'scraping_list' notebook to create dataframe.
#dataframe index is game name, 

games = {}

for row in rows:
    title = (row.find('span', class_='title').text)
    url = row.get('href')
    release_date = (row.find('div', class_='col search_released responsive_secondrow').text)
    if (row.find('div', class_='col search_discount responsive_secondrow').text.strip()) == '':
        been_sale = 0
    else:
        been_sale = 1
    if been_sale == 0:
        orig_price = (row.find('div', class_='col search_price responsive_secondrow').text.strip())
    else:
        orig_price = (row.find('strike').text)
    games[title] = (url, 
                    release_date,
                    been_sale,
                    orig_price)
    

In [56]:
gamesdf1 = pd.DataFrame(games).T
gamesdf1.columns = ['link', 'release_date', 'been_sale', 'orig_price']

gamesdf1.shape

(16560, 4)

In [57]:
#no nulls!!
gamesdf1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16560 entries, OUTRIDERS to Dinosaur Hunt - Stegosaurus Expansion Pack
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   link          16560 non-null  object
 1   release_date  16560 non-null  object
 2   been_sale     16560 non-null  object
 3   orig_price    16560 non-null  object
dtypes: object(4)
memory usage: 646.9+ KB


In [58]:
gamesdf1.head()

,link,release_date,been_sale,orig_price
OUTRIDERS,https://store.steampowered.com/sub/504022/?snr...,"Apr 1, 2021",0,$59.99
Sea of Thieves,https://store.steampowered.com/app/1172620/Sea...,"Jun 3, 2020",1,$39.99
Halo: The Master Chief Collection,https://store.steampowered.com/sub/376282/?snr...,"Dec 3, 2019",1,$39.99
It Takes Two,https://store.steampowered.com/app/1426210/It_...,"Mar 26, 2021",0,$39.99
Valheim,https://store.steampowered.com/app/892970/Valh...,"Feb 2, 2021",0,$19.99


In [64]:
gamesdf1.tail()

,link,release_date,been_sale,orig_price
Dinosaur Hunt - Dragon Hunter Expansion Pack,https://store.steampowered.com/app/408102/Dino...,"Dec 18, 2015",0,$0.99
Dinosaur Hunt - Giant Spiders Hunter Expansion Pack,https://store.steampowered.com/app/431191/Dino...,"Mar 16, 2016",0,$0.99
Dinosaur Hunt - Brontosaurus Expansion Pack,https://store.steampowered.com/app/434942/Dino...,"Mar 16, 2016",0,$0.99
"Dinosaur Hunt - Vampires, Gargoyles, Mutants Hunter Expansion Pack",https://store.steampowered.com/app/431190/Dino...,"Jan 22, 2016",0,$0.99
Dinosaur Hunt - Stegosaurus Expansion Pack,https://store.steampowered.com/app/408100/Dino...,"Nov 13, 2015",0,$0.99


# attempt 1 to pull all game page data

Just "successfully" ran the function to pull all game data for the first 50 games. several nulls. going to run the code on here overnight, to see if there are enough non-null games out of the 16k to run a suitable model. if not, will have to write conditions for variance in page layouts, or supplement my scraped data with steamspy/steam's api

In [69]:
#attempting to pull my code in from  'scraping_page' to see if data can be pulled from from each game page.

def get_game_details(url):

    

    
    
    #Request HTML and parse
    response = driver.get(url)
    
    for i in range(5):
        #Scroll
        driver.execute_script(
            "window.scrollTo(0, document.documentElement.scrollHeight);" #Alternatively, document.body.scrollHeight
        )
        time.sleep(1)
    
    soup = BeautifulSoup(driver.page_source, "html5lib")
    
    

    
    headers = ['title', 'num_reviews', 'review_rating', 
              'num_languages', 'tags', 'num_tags',
              'genres', 'num_genres', 'publisher']
    
    #Get title
    if not (soup.find('div', class_='apphub_AppName')):
        title = 'HELP'
    else:
        title = soup.find('div', class_='apphub_AppName').text

    #Get number of reviews
    if not (soup.find('div', id='Reviews_summary')):
        num_reviews = 'HELP'
    elif not (soup.find('div', id='Reviews_summary').find('div', id='ViewAllReviewssummary')):
        num_reviews = 'HELP'
    else:
        rev_string = soup.find('div', id='Reviews_summary').find('div', id='ViewAllReviewssummary').find('a').text
        num_reviews = rev_string.split()[2]
     

    #Get review rating
    if not (soup.find(class_='user_reviews_summary_bar')):
        rating = 'HELP'
    else:
        rating = soup.find(class_='user_reviews_summary_bar').find_all('span')[0].text
    
    
    #Get number of languages offered in
    if not soup.find('a', class_='all_languages'):
        num_languages = 'HELP'
    else:
        num_languages = soup.find('a', class_='all_languages').text.split()[2]
    

    #Get tags associated with game
    if not soup.find('div', class_='glance_tags'):
        tags_list = 'HELP'
    else:
        tags = soup.find('div', class_='glance_tags').find_all('a')
        tags_list = []
        for items in tags:
            tags_list.append(items.text.strip())
    
    
    #number of tags
    if not soup.find('div', class_='glance_tags'):
        num_tags = 'HELP'
    else:
        num_tags = len(tags_list)
    
    #genres list
    if not soup.find('div', class_='details_block'):
        genres_list = 'HELP'
    else:
        genres_all = soup.find('div', class_='details_block').find_all('a')[0:-2]
        genres_list = []
        for links in genres_all:
            genres_list.append(links.text)
        
    # number of genres
    if not soup.find('div', class_='details_block'):
        num_genres = 'HELP'
    else:
        num_genres = len(genres_list)
    
    #get publisher
    
    #if not soup.find('div', class_='details_block'): #this threw an error 10 pages in, going more general
    #    publisher = 'HALP PUBLISHER'
    #else:
    #    publisher = soup.find('div', class_='details_block').find_all('a')[-1].text
    
    
    if not soup.find('div', class_='glance_ctn_responsive_left'):
        publisher = 'HELP'
    else:
        publisher = soup.find('div', class_='glance_ctn_responsive_left').find_all('a')[-1].text
    
    #create dict of all games
    games_dict = dict(zip(headers, [title,
                                num_reviews,
                                rating,
                                num_languages, 
                                tags_list,
                                num_tags,
                                genres_list,
                                num_genres,
                                publisher]))

    return games_dict

In [70]:
games_info = []

for url in gamesdf1.link:
    games_info.append(get_game_details(url))
    

AttributeError: 'NoneType' object has no attribute 'text'

In [77]:
games_info = pd.DataFrame(games_info)

In [73]:
gamesdf1.iloc[312]

link            https://store.steampowered.com/app/10090/Call_...
release_date                                         Nov 18, 2008
been_sale                                                       0
orig_price                                                 $19.99
Name: Call of Duty: World at War, dtype: object

In [ ]:
# first attempt failed, but only after 300 iterations. going to look into nulls now, but am going
# to rewrite the function to account for failing at EVERY find, not just first one

In [80]:
import numpy as np
games_info.replace('HELP', np.nan, inplace=True)

In [81]:
games_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 311 entries, 0 to 310
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   title          257 non-null    object 
 1   num_reviews    207 non-null    object 
 2   review_rating  246 non-null    object 
 3   num_languages  197 non-null    object 
 4   tags           256 non-null    object 
 5   num_tags       256 non-null    float64
 6   genres         310 non-null    object 
 7   num_genres     310 non-null    float64
 8   publisher      256 non-null    object 
dtypes: float64(2), object(7)
memory usage: 22.0+ KB


# attempt #2
ran as I went to sleep, computer was asleep when I woke up, woke it up, program still running, assumed that it was running all night, but had to leave for bridal shower. terminated program, only ~400 titles copied. need to remove the sleep, takes too long between pages. although first time i tried, it it wasnt always scrolling to the bottom of the game page. will play with it tomorrow. 

In [83]:
#attempting to pull my code in from  'scraping_page' to see if data can be pulled from from each game page.

def get_game_details(url):

    

    
    
    #Request HTML and parse
    response = driver.get(url)
    
    for i in range(5):
        #Scroll
        driver.execute_script(
            "window.scrollTo(0, document.documentElement.scrollHeight);" #Alternatively, document.body.scrollHeight
        )
        time.sleep(1)
    
    soup = BeautifulSoup(driver.page_source, "html5lib")
    
    

    
    headers = ['title', 'num_reviews', 'review_rating', 
              'num_languages', 'tags', 'num_tags',
              'genres', 'num_genres', 'publisher']
    
    #Get title
    if not (soup.find('div', class_='apphub_AppName')):
        title = 'HELP'
    else:
        title = soup.find('div', class_='apphub_AppName').text

    #Get number of reviews
    if not (soup.find('div', id='Reviews_summary')):
        num_reviews = 'HELP'
    elif not (soup.find('div', id='Reviews_summary').find('div', id='ViewAllReviewssummary')):
        num_reviews = 'HELP'
    elif not soup.find('div', id='Reviews_summary').find('div', id='ViewAllReviewssummary').find('a'):
        num_reviews = 'HELP'
    else:
        rev_string = soup.find('div', id='Reviews_summary').find('div', id='ViewAllReviewssummary').find('a').text
        num_reviews = rev_string.split()[2]
     

    #Get review rating
    if not (soup.find(class_='user_reviews_summary_bar')):
        rating = 'HELP'
    elif not soup.find(class_='user_reviews_summary_bar').find_all('span'):
        rating = 'HELP'
    else:
        rating = soup.find(class_='user_reviews_summary_bar').find_all('span')[0].text
    
    
    #Get number of languages offered in
    if not soup.find('a', class_='all_languages'):
        num_languages = 'HELP'
    else:
        num_languages = soup.find('a', class_='all_languages').text.split()[2]
    

    #Get tags associated with game
    if not soup.find('div', class_='glance_tags'):
        tags_list = 'HELP'
    else:
        tags = soup.find('div', class_='glance_tags').find_all('a')
        tags_list = []
        for items in tags:
            tags_list.append(items.text.strip())
    
    
    #number of tags
    if not soup.find('div', class_='glance_tags'):
        num_tags = 'HELP'
    else:
        num_tags = len(tags_list)
    
    #genres list
    if not soup.find('div', class_='details_block'):
        genres_list = 'HELP'
    else:
        genres_all = soup.find('div', class_='details_block').find_all('a')[0:-2]
        genres_list = []
        for links in genres_all:
            genres_list.append(links.text)
        
    # number of genres
    if not soup.find('div', class_='details_block'):
        num_genres = 'HELP'
    else:
        num_genres = len(genres_list)
    
    #get publisher
    
    #if not soup.find('div', class_='details_block'): #this threw an error 10 pages in, going more general
    #    publisher = 'HALP PUBLISHER'
    #else:
    #    publisher = soup.find('div', class_='details_block').find_all('a')[-1].text
    
    
    if not soup.find('div', class_='glance_ctn_responsive_left'):
        publisher = 'HELP'
    else:
        publisher = soup.find('div', class_='glance_ctn_responsive_left').find_all('a')[-1].text
    
    #create dict of all games
    games_dict = dict(zip(headers, [title,
                                num_reviews,
                                rating,
                                num_languages, 
                                tags_list,
                                num_tags,
                                genres_list,
                                num_genres,
                                publisher]))

    return games_dict

In [84]:
games_info = []

for url in gamesdf1.link:
    games_info.append(get_game_details(url))

WebDriverException: Message: disconnected: Unable to receive message from renderer
  (Session info: chrome=89.0.4389.114)


In [85]:
len(games_info)

406

In [87]:
games_info = pd.DataFrame(games_info)

In [93]:
games_info.replace(np.nan, null)

NameError: name 'null' is not defined

In [90]:
games_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 406 entries, 0 to 405
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   title          406 non-null    object
 1   num_reviews    406 non-null    object
 2   review_rating  406 non-null    object
 3   num_languages  406 non-null    object
 4   tags           406 non-null    object
 5   num_tags       406 non-null    object
 6   genres         406 non-null    object
 7   num_genres     406 non-null    object
 8   publisher      406 non-null    object
dtypes: object(9)
memory usage: 28.7+ KB


In [94]:
games_info.head()

,title,num_reviews,review_rating,num_languages,tags,num_tags,genres,num_genres,publisher
0,HELP,HELP,HELP,HELP,HELP,HELP,"[Action, Adventure, RPG, People Can Fly]",4,HELP
1,Sea of Thieves,"109,916",Very Positive,13,"[Adventure, Multiplayer, Open World, Pirates, ...",20,"[Action, Adventure]",2,Xbox Game Studios
2,HELP,HELP,HELP,HELP,HELP,HELP,"[Action, 343 Industries, Splash Damage, Ruffia...",8,HELP
3,It Takes Two,"10,433",Overwhelmingly Positive,12,"[Co-op, Adventure, 3D Platformer, Puzzle, Puzz...",20,"[Action, Adventure, Hazelight]",3,Electronic Arts
4,Valheim,"215,071",Overwhelmingly Positive,12,"[Open World Survival Craft, Online Co-Op, Open...",20,"[Action, Adventure, Indie, RPG, Early Access, ...",6,Coffee Stain Publishing


# attempt number 3 

going to add to the scoll, but take awawy the sleep, hoping that it will cycle through the site faster

In [99]:
#attempting to pull my code in from  'scraping_page' to see if data can be pulled from from each game page.

def get_game_details(url):

    

    
    
    #Request HTML and parse
    response = driver.get(url)
    
    for i in range(7):
        #Scroll
        driver.execute_script(
            "window.scrollTo(0, document.documentElement.scrollHeight);" #Alternatively, document.body.scrollHeight
        )
        #time.sleep(1)
    
    soup = BeautifulSoup(driver.page_source, "html5lib")
    
    

    
    headers = ['title', 'num_reviews', 'review_rating', 
              'num_languages', 'tags', 'num_tags',
              'genres', 'num_genres', 'publisher']
    
    #Get title
    if not (soup.find('div', class_='apphub_AppName')):
        title = 'HELP'
    else:
        title = soup.find('div', class_='apphub_AppName').text

    #Get number of reviews
    if not (soup.find('div', id='Reviews_summary')):
        num_reviews = 'HELP'
    elif not (soup.find('div', id='Reviews_summary').find('div', id='ViewAllReviewssummary')):
        num_reviews = 'HELP'
    elif not soup.find('div', id='Reviews_summary').find('div', id='ViewAllReviewssummary').find('a'):
        num_reviews = 'HELP'
    else:
        rev_string = soup.find('div', id='Reviews_summary').find('div', id='ViewAllReviewssummary').find('a').text
        num_reviews = rev_string.split()[2]
     

    #Get review rating
    if not (soup.find(class_='user_reviews_summary_bar')):
        rating = 'HELP'
    elif not soup.find(class_='user_reviews_summary_bar').find_all('span'):
        rating = 'HELP'
    else:
        rating = soup.find(class_='user_reviews_summary_bar').find_all('span')[0].text
    
    
    #Get number of languages offered in
    if not soup.find('a', class_='all_languages'):
        num_languages = 'HELP'
    else:
        num_languages = soup.find('a', class_='all_languages').text.split()[2]
    

    #Get tags associated with game
    if not soup.find('div', class_='glance_tags'):
        tags_list = 'HELP'
    else:
        tags = soup.find('div', class_='glance_tags').find_all('a')
        tags_list = []
        for items in tags:
            tags_list.append(items.text.strip())
    
    
    #number of tags
    if not soup.find('div', class_='glance_tags'):
        num_tags = 'HELP'
    else:
        num_tags = len(tags_list)
    
    #genres list
    if not soup.find('div', class_='details_block'):
        genres_list = 'HELP'
    else:
        genres_all = soup.find('div', class_='details_block').find_all('a')[0:-2]
        genres_list = []
        for links in genres_all:
            genres_list.append(links.text)
        
    # number of genres
    if not soup.find('div', class_='details_block'):
        num_genres = 'HELP'
    else:
        num_genres = len(genres_list)
    
    #get publisher
    
    #if not soup.find('div', class_='details_block'): #this threw an error 10 pages in, going more general
    #    publisher = 'HALP PUBLISHER'
    #else:
    #    publisher = soup.find('div', class_='details_block').find_all('a')[-1].text
    
    
    if not soup.find('div', class_='glance_ctn_responsive_left'):
        publisher = 'HELP'
    else:
        publisher = soup.find('div', class_='glance_ctn_responsive_left').find_all('a')[-1].text
    
    #create dict of all games
    games_dict = dict(zip(headers, [title,
                                num_reviews,
                                rating,
                                num_languages, 
                                tags_list,
                                num_tags,
                                genres_list,
                                num_genres,
                                publisher]))

    return games_dict

In [100]:
games_info = []

for url in gamesdf1.link:
    games_info.append(get_game_details(url))

KeyboardInterrupt: 

In [102]:
len(games_info)

46

In [ ]:
'''
SO, stopped after 3/5 minutes. it would go through 3/4 pages lightning fast, then would pause for a while to
load. could be deanas internet, have never worked here before. watching the pages, it did not appear that the
pages had enough time (when not loading) to get to the bottom which is where the total numbr of reviews comes in.
going to see number of num reviews pulled from this test.
MOVING FORWARD i will save 2 dfs at a time, post and current, so I can compare to last iteration. should
have thought of that already.
'''

In [104]:
games_info = pd.DataFrame(games_info)

In [108]:
games_info.head(1)

,title,num_reviews,review_rating,num_languages,tags,num_tags,genres,num_genres,publisher
0,HELP,HELP,HELP,HELP,HELP,HELP,"[Action, Adventure, RPG, People Can Fly]",4,HELP


In [106]:
prev_df = games_info.copy()

# attempt 4
inverse attempt, fewer scrolls, but leaving the sleep, 

In [152]:
#attempting to pull my code in from  'scraping_page' to see if data can be pulled from from each game page.

def get_game_details(url):

    

    
    
    #Request HTML and parse
    response = driver.get(url)
    
    for i in range(3):
        #Scroll
        driver.execute_script(
            "window.scrollTo(0, document.documentElement.scrollHeight);" #Alternatively, document.body.scrollHeight
        )
        time.sleep(1)
    
    soup = BeautifulSoup(driver.page_source, "html5lib")
    
    

    
    headers = ['title', 'num_reviews', 'review_rating', 
              'num_languages', 'tags', 'num_tags',
              'genres', 'num_genres', 'publisher']
    
    #Get title
    if not (soup.find('div', class_='apphub_AppName')):
        title = 'HELP'
    else:
        title = soup.find('div', class_='apphub_AppName').text

    #Get number of reviews
    if not (soup.find('div', id='Reviews_summary')):
        num_reviews = 'HELP'
    elif not (soup.find('div', id='Reviews_summary').find('div', id='ViewAllReviewssummary')):
        num_reviews = 'HELP'
    elif not soup.find('div', id='Reviews_summary').find('div', id='ViewAllReviewssummary').find('a'):
        num_reviews = 'HELP'
    else:
        rev_string = soup.find('div', id='Reviews_summary').find('div', id='ViewAllReviewssummary').find('a').text
        num_reviews = rev_string.split()[2]
     

    #Get review rating
    if not (soup.find(class_='user_reviews_summary_bar')):
        rating = 'HELP'
    elif not soup.find(class_='user_reviews_summary_bar').find_all('span'):
        rating = 'HELP'
    else:
        rating = soup.find(class_='user_reviews_summary_bar').find_all('span')[0].text
    
    
    #Get number of languages offered in
    if not soup.find('a', class_='all_languages'):
        num_languages = 'HELP'
    else:
        num_languages = soup.find('a', class_='all_languages').text.split()[2]
    

    #Get tags associated with game
    if not soup.find('div', class_='glance_tags'):
        tags_list = 'HELP'
    else:
        tags = soup.find('div', class_='glance_tags').find_all('a')
        tags_list = []
        for items in tags:
            tags_list.append(items.text.strip())
    
    
    #number of tags
    if not soup.find('div', class_='glance_tags'):
        num_tags = 'HELP'
    else:
        num_tags = len(tags_list)
    
    #genres list
    if not soup.find('div', class_='details_block'):
        genres_list = 'HELP'
    else:
        genres_all = soup.find('div', class_='details_block').find_all('a')[0:-2]
        genres_list = []
        for links in genres_all:
            genres_list.append(links.text)
        
    # number of genres
    if not soup.find('div', class_='details_block'):
        num_genres = 'HELP'
    else:
        num_genres = len(genres_list)
    
    #get publisher
    
    #if not soup.find('div', class_='details_block'): #this threw an error 10 pages in, going more general
    #    publisher = 'HALP PUBLISHER'
    #else:
    #    publisher = soup.find('div', class_='details_block').find_all('a')[-1].text
    
    
    if not soup.find('div', class_='glance_ctn_responsive_left'):
        publisher = 'HELP'
    else:
        publisher = soup.find('div', class_='glance_ctn_responsive_left').find_all('a')[-1].text
    
    #create dict of all games
    games_dict = dict(zip(headers, [title,
                                num_reviews,
                                rating,
                                num_languages, 
                                tags_list,
                                num_tags,
                                genres_list,
                                num_genres,
                                publisher]))

    return games_dict

In [153]:
games_info = []

for url in gamesdf1.link:
    games_info.append(get_game_details(url))


KeyboardInterrupt: 

In [120]:
games_info_df = pd.DataFrame(games_info)

In [121]:
games_info_df.shape

(1068, 9)

In [123]:
games_info_df.tail()

,title,num_reviews,review_rating,num_languages,tags,num_tags,genres,num_genres,publisher
1063,Workers & Resources: Soviet Republic,"7,876",Very Positive,10,"[City Builder, Economy, Strategy, Simulation, ...",20,"[Indie, Simulation, Strategy, Early Access]",4,3Division
1064,HELP,HELP,HELP,HELP,HELP,HELP,"[Indie, RPG, Strategy]",3,HELP
1065,Hunt: Showdown - Ronin,HELP,Very Positive,11,"[Action, Western, Zombies, Shooter, Blood, Hun...",8,[],0,Crytek
1066,The Sims™ 4 Bowling Night Stuff,HELP,4 user reviews,18,[Simulation],1,"[Simulation, Maxis]",2,Electronic Arts
1067,HELP,HELP,HELP,HELP,HELP,HELP,"[Action, Adventure, Crystal Dynamics, Crystal ...",15,HELP


In [124]:
games_info_df.replace('HELP', np.nan, inplace=True)

In [127]:
games_info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1068 entries, 0 to 1067
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   title          859 non-null    object 
 1   num_reviews    626 non-null    object 
 2   review_rating  833 non-null    object 
 3   num_languages  617 non-null    object 
 4   tags           858 non-null    object 
 5   num_tags       858 non-null    float64
 6   genres         1067 non-null   object 
 7   num_genres     1067 non-null   float64
 8   publisher      858 non-null    object 
dtypes: float64(2), object(7)
memory usage: 75.2+ KB


In [129]:
prev_df = games_info_df.copy()

In [214]:
#attempting to pull my code in from  'scraping_page' to see if data can be pulled from from each game page.

def get_game_details(url):

    

    
    
    #Request HTML and parse
    response = driver.get(url)
    
    for i in range(5):
        #Scroll
        driver.execute_script(
            "window.scrollTo(0, document.documentElement.scrollHeight);" #Alternatively, document.body.scrollHeight
        )
        time.sleep(.2)
    
    soup = BeautifulSoup(driver.page_source, "html5lib")
    
    

    
    headers = ['title', 'num_reviews', 'review_rating', 
              'num_languages', 'tags', 'num_tags',
              'genres', 'num_genres', 'publisher']
    
    #Get title
    if not (soup.find('div', class_='apphub_AppName')):
        title = 'HELP'
    else:
        title = soup.find('div', class_='apphub_AppName').text

    #Get number of reviews
    if not (soup.find('div', id='Reviews_summary')):
        num_reviews = 'HELP'
    elif not (soup.find('div', id='Reviews_summary').find('div', id='ViewAllReviewssummary')):
        num_reviews = 'HELP'
    elif not soup.find('div', id='Reviews_summary').find('div', id='ViewAllReviewssummary').find('a'):
        num_reviews = 'HELP'
    else:
        rev_string = soup.find('div', id='Reviews_summary').find('div', id='ViewAllReviewssummary').find('a').text
        num_reviews = rev_string.split()[2]
     

    #Get review rating
    if not (soup.find(class_='user_reviews_summary_bar')):
        rating = 'HELP'
    elif not soup.find(class_='user_reviews_summary_bar').find_all('span'):
        rating = 'HELP'
    else:
        rating = soup.find(class_='user_reviews_summary_bar').find_all('span')[0].text
    
    
    #Get number of languages offered in
    if not soup.find('a', class_='all_languages'):
        num_languages = 'HELP'
    else:
        num_languages = soup.find('a', class_='all_languages').text.split()[2]
    

    #Get tags associated with game
    if not soup.find('div', class_='glance_tags'):
        tags_list = 'HELP'
    else:
        tags = soup.find('div', class_='glance_tags').find_all('a')
        tags_list = []
        for items in tags:
            tags_list.append(items.text.strip())
    
    
    #number of tags
    if not soup.find('div', class_='glance_tags'):
        num_tags = 'HELP'
    else:
        num_tags = len(tags_list)
    
    #genres list
    if not soup.find('div', class_='details_block'):
        genres_list = 'HELP'
    else:
        genres_all = soup.find('div', class_='details_block').find_all('a')[0:-2]
        genres_list = []
        for links in genres_all:
            genres_list.append(links.text)
        
    # number of genres
    if not soup.find('div', class_='details_block'):
        num_genres = 'HELP'
    else:
        num_genres = len(genres_list)
    
    #get publisher
    
    #if not soup.find('div', class_='details_block'): #this threw an error 10 pages in, going more general
    #    publisher = 'HALP PUBLISHER'
    #else:
    #    publisher = soup.find('div', class_='details_block').find_all('a')[-1].text
    
    
    if not soup.find('div', class_='glance_ctn_responsive_left'):
        publisher = 'HELP'
    elif not soup.find('div',class_='glance_ctn_responsive_left').find_all('a'):
        publisher = 'HELP'
    else:
        publisher = soup.find('div', class_='glance_ctn_responsive_left').find_all('a')[-1].text
    
    #create dict of all games
    games_dict = dict(zip(headers, [title,
                                num_reviews,
                                rating,
                                num_languages, 
                                tags_list,
                                num_tags,
                                genres_list,
                                num_genres,
                                publisher]))

    return games_dict

In [215]:
games_info = []

for url in gamesdf1.link[2525:]:
    games_info.append(get_game_details(url))



In [ ]:
len(games_df1)

In [216]:
len(games_info)

14035

In [177]:
games1 = pd.DataFrame(games_info)

In [179]:
games1.replace('HELP', np.nan, inplace=True)

In [181]:
games1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2104 entries, 0 to 2103
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   title          1707 non-null   object 
 1   num_reviews    1131 non-null   object 
 2   review_rating  1672 non-null   object 
 3   num_languages  1195 non-null   object 
 4   tags           1706 non-null   object 
 5   num_tags       1706 non-null   float64
 6   genres         2102 non-null   object 
 7   num_genres     2102 non-null   float64
 8   publisher      1706 non-null   object 
dtypes: float64(2), object(7)
memory usage: 148.1+ KB


In [ ]:
'''
ok by now we made the scraping speed as besst as possible. stopped when computer went to sleep, 
completed about 2k, unfortuantely not all sites have that total reviews count. will run again starting at
last game completed, amd look for supplementary df somewhere els
'''

In [190]:
gamesdf1.iloc[]

link            https://store.steampowered.com/sub/40569/?snr=...
release_date                                         Jun 23, 2014
been_sale                                                       0
orig_price                                                 $19.99
Name: COH 2 - The Western Front Armies (Double Pack), dtype: object

In [187]:
games1.iloc[-2]

title                             Warhammer 40,000: Dawn of War II
num_reviews                                                  8,784
review_rating                                        Very Positive
num_languages                                                  NaN
tags             [Warhammer 40K, Strategy, RTS, Sci-fi, Multipl...
num_tags                                                        20
genres           [Strategy, Relic Entertainment, Feral Interact...
num_genres                                                       3
publisher                            Feral Interactive (Mac/Linux)
Name: 2102, dtype: object

In [188]:
gamesdf1.loc['Warhammer 40,000: Dawn of War II']

link            https://store.steampowered.com/app/15620/Warha...
release_date                                         Feb 18, 2009
been_sale                                                       0
orig_price                                                 $19.99
Name: Warhammer 40,000: Dawn of War II, dtype: object

In [195]:
games1.drop_duplicates(subest='title')

TypeError: drop_duplicates() got an unexpected keyword argument 'subest'

In [204]:
len(games_info)

421

In [205]:
games2 = pd.DataFrame(games_info)

In [206]:
games2

,title,num_reviews,review_rating,num_languages,tags,num_tags,genres,num_genres,publisher
0,MechWarrior Online™ Solaris 7 Hero Pack,HELP,Positive,HELP,"[Strategy, Simulation, Action, Free to Play, M...",6,"[Action, Free to Play, Massively Multiplayer, ...",6,Piranha Games Inc.
1,PAYDAY 2: Federales Weapon Pack,HELP,Mostly Positive,9,"[RPG, Action, Violent, Gore]",4,"[Action, RPG]",2,Starbreeze Publishing AB
2,"Warhammer 40,000: Dawn of War II","8,784",Very Positive,HELP,"[Warhammer 40K, Strategy, RTS, Sci-fi, Multipl...",20,"[Strategy, Relic Entertainment, Feral Interact...",3,Feral Interactive (Mac/Linux)
3,HELP,HELP,HELP,HELP,HELP,HELP,"[Simulation, Strategy, Paradox Development Stu...",3,HELP
4,Artemis Spaceship Bridge Simulator,433,Very Positive,HELP,"[Simulation, Indie, Action, Space, Co-op, Mult...",8,[Action],1,HELP
...,...,...,...,...,...,...,...,...,...
416,THE MONSTER OF MEMORY:DESTINY,79,Very Positive,HELP,"[Visual Novel, LGBTQ+, Dating Sim, Casual, Wor...",13,"[Adventure, Casual, Narrator]",3,Tong Li Publishing Co.
417,Tower!3D Pro,647,Mixed,HELP,"[Simulation, Flight, Management, Voice Control...",6,[Simulation],1,FeelThere
418,PixARK,"8,108",Mostly Positive,13,"[Survival, Adventure, Massively Multiplayer, O...",20,"[Action, Adventure, Casual, Massively Multipla...",6,Snail Games USA
419,Laid-Back Camp - Virtual - Fumoto Campsite,HELP,Positive,6,"[Choose Your Own Adventure, Visual Novel, Poin...",19,[],0,Clouded Leopard Entertainment Inc.


In [210]:
games_joined = pd.concat([games1, games2])

In [212]:
games_joined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2525 entries, 0 to 420
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   title          2128 non-null   object 
 1   num_reviews    1552 non-null   object 
 2   review_rating  2093 non-null   object 
 3   num_languages  1616 non-null   object 
 4   tags           2127 non-null   object 
 5   num_tags       2127 non-null   object 
 6   genres         2523 non-null   object 
 7   num_genres     2523 non-null   float64
 8   publisher      2127 non-null   object 
dtypes: float64(1), object(8)
memory usage: 197.3+ KB


In [218]:
games3 = pd.DataFrame(games_info)

In [219]:
games3.replace('HELP', np.nan, inplace=True)

In [220]:
games3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14035 entries, 0 to 14034
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   title          12951 non-null  object 
 1   num_reviews    4192 non-null   object 
 2   review_rating  11914 non-null  object 
 3   num_languages  5359 non-null   object 
 4   tags           12951 non-null  object 
 5   num_tags       12951 non-null  float64
 6   genres         14021 non-null  object 
 7   num_genres     14021 non-null  float64
 8   publisher      12937 non-null  object 
dtypes: float64(2), object(7)
memory usage: 987.0+ KB


In [222]:
games_joined = pd.concat([games_joined, games3])

In [224]:
games_joined.shape

(16560, 9)

In [225]:
games_joined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16560 entries, 0 to 14034
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   title          15079 non-null  object 
 1   num_reviews    5744 non-null   object 
 2   review_rating  14007 non-null  object 
 3   num_languages  6975 non-null   object 
 4   tags           15078 non-null  object 
 5   num_tags       15078 non-null  object 
 6   genres         16544 non-null  object 
 7   num_genres     16544 non-null  float64
 8   publisher      15064 non-null  object 
dtypes: float64(1), object(8)
memory usage: 1.3+ MB


# update
was able to successfully scrape all the pages from the list. took about 10 hours had it running while sleeping. UNFORTUNATELY oonly 5k of 16k had the number of reviews on the page where i thought it would be. can use these numbers for preliminary modeling, but i have to go back and see if ther eis another place that has total revies. its easy to find how many reviews mating the ranking listed, but that number is different than that listed for total. will check steam spy or other pages, but this was a good test. if I didnt have the shower this weekend I would have an etra day to work on it, but the mvp is not due until tomorrow so i have one more day to work it out

In [226]:
# have to merge the 2 tables, so i can use date, sale, and orig price in modeling


In [227]:
gamesdf1.head(2)

,link,release_date,been_sale,orig_price
OUTRIDERS,https://store.steampowered.com/sub/504022/?snr...,"Apr 1, 2021",0,$59.99
Sea of Thieves,https://store.steampowered.com/app/1172620/Sea...,"Jun 3, 2020",1,$39.99


In [228]:
gamesdf1.shape

(16560, 4)

In [234]:
totaldf = pd.merge(gamesdf1, games_joined, left_index=True, right_index=True)

In [230]:
games_joined.head(2)

,title,num_reviews,review_rating,num_languages,tags,num_tags,genres,num_genres,publisher
0,NaN,NaN,NaN,NaN,NaN,NaN,"[Action, Adventure, RPG, People Can Fly]",4.0,NaN
1,Sea of Thieves,"111,078",Very Positive,13,"[Adventure, Multiplayer, Open World, Pirates, ...",20,"[Action, Adventure]",2.0,Xbox Game Studios


In [231]:
games_joined.shape

(16560, 9)

In [235]:
totaldf.head

<bound method NDFrame.head of Empty DataFrame
Columns: [link, release_date, been_sale, orig_price, title, num_reviews, review_rating, num_languages, tags, num_tags, genres, num_genres, publisher]
Index: []>

In [238]:
gamesdf1reset = gamesdf1.reset_index()

In [242]:
totaldf = pd.merge(gamesdf1reset ,games_joined, left_index=True, right_index=True)

In [251]:
totaldf.shape

(16560, 14)

In [256]:
totaldf.drop_duplicates(subset='index', inplace=True)

In [257]:
totaldf.shape

(14035, 14)

In [264]:
totaldf.tail()

,index,link,release_date,been_sale,orig_price,title,num_reviews,review_rating,num_languages,tags,num_tags,genres,num_genres,publisher
14030,Yatzy,https://store.steampowered.com/app/749010/Yatz...,"Nov 21, 2017",0,$4.99,Dinosaur Hunt - Dragon Hunter Expansion Pack,NaN,Mostly Positive,NaN,"[Adventure, Indie, Simulation]",3,"[Action, Adventure, Indie, Simulation]",4.0,Racing Bros
14031,Zombies on a Plane,https://store.steampowered.com/app/297450/Zomb...,"Mar 25, 2016",0,$4.99,Dinosaur Hunt - Giant Spiders Hunter Expansion...,NaN,Mixed,NaN,"[Adventure, Indie, Simulation]",3,"[Action, Adventure, Indie, Simulation]",4.0,Racing Bros
14032,CTHON,https://store.steampowered.com/app/595100/CTHO...,"Mar 17, 2017",0,$4.99,Dinosaur Hunt - Brontosaurus Expansion Pack,NaN,Mixed,NaN,"[Adventure, Indie, Simulation, Action]",4,"[Action, Adventure, Indie, Simulation]",4.0,Racing Bros
14033,Awesomenauts - Dark Masquerade Qi'Tara Skin,https://store.steampowered.com/app/634014/Awes...,"Dec 6, 2017",0,$4.99,"Dinosaur Hunt - Vampires, Gargoyles, Mutants H...",NaN,Mostly Positive,NaN,"[Adventure, Indie, Simulation]",3,"[Action, Adventure, Indie, Simulation]",4.0,Racing Bros
14034,Monster Truck Destruction,https://store.steampowered.com/app/324760/Mons...,"Jul 10, 2015",0,$4.99,Dinosaur Hunt - Stegosaurus Expansion Pack,NaN,Mostly Positive,NaN,"[Adventure, Indie, Simulation]",3,"[Action, Adventure, Indie, Simulation]",4.0,Racing Bros


In [263]:
totaldf.head()

,index,link,release_date,been_sale,orig_price,title,num_reviews,review_rating,num_languages,tags,num_tags,genres,num_genres,publisher
0,OUTRIDERS,https://store.steampowered.com/sub/504022/?snr...,"Apr 1, 2021",0,$59.99,NaN,NaN,NaN,NaN,NaN,NaN,"[Action, Adventure, RPG, People Can Fly]",4.0,NaN
1,Sea of Thieves,https://store.steampowered.com/app/1172620/Sea...,"Jun 3, 2020",1,$39.99,Sea of Thieves,"111,078",Very Positive,13,"[Adventure, Multiplayer, Open World, Pirates, ...",20,"[Action, Adventure]",2.0,Xbox Game Studios
2,Halo: The Master Chief Collection,https://store.steampowered.com/sub/376282/?snr...,"Dec 3, 2019",1,$39.99,NaN,NaN,NaN,NaN,NaN,NaN,"[Action, 343 Industries, Splash Damage, Ruffia...",8.0,NaN
3,It Takes Two,https://store.steampowered.com/app/1426210/It_...,"Mar 26, 2021",0,$39.99,It Takes Two,"11,813",Overwhelmingly Positive,12,"[Co-op, Adventure, 3D Platformer, Puzzle, Puzz...",20,"[Action, Adventure, Hazelight]",3.0,Electronic Arts
4,Valheim,https://store.steampowered.com/app/892970/Valh...,"Feb 2, 2021",0,$19.99,Valheim,"216,651",Overwhelmingly Positive,12,"[Open World Survival Craft, Online Co-Op, Surv...",20,"[Action, Adventure, Indie, RPG, Early Access, ...",6.0,Coffee Stain Publishing


In [ ]:
# merged, but looking at inex(title from first df) and title (title form second df) not all are equal.
# need to re-merge on title. checking both tails of dfs to 